## FaBo AI Car BoardのGPIO Test

Jetson GPIOバージョンチェック

In [ ]:
!pip3 show Jetson.GPIO

GPIO出力ピンのチェック　LEDに繋ぐ場合は、必ずトランジスタかMOSーFETで電流を増幅させる。

In [ ]:
import Jetson.GPIO as GPIO
import time

# 物理ピン マッピング
pins = [12, 16, 18, 22, 32, 36, 38 ,40]

# GPIO 初期化
GPIO.setmode(GPIO.BOARD)
for pin in pins:
    GPIO.setup(pin, GPIO.OUT)

try:
    while True:
        # 点灯
        for pin in pins:
            GPIO.output(pin, GPIO.HIGH)
        time.sleep(0.5)

        # 消灯
        for pin in pins:
            GPIO.output(pin, GPIO.LOW)
        time.sleep(0.5)

except KeyboardInterrupt:
    print("終了中...")

finally:
    GPIO.cleanup()


入力ピンのテスト　ボタンを用意します。入力電圧は3.3V~2.0Vで必ず入力します。

In [ ]:
import Jetson.GPIO as GPIO
import time
from IPython.display import display, clear_output, Markdown

# 物理ピンと BCM GPIO 番号のマッピング
pin_map = {
    11: 17,
    13: 27,
    15: 22,
    29: 5,
    31: 6,
    33: 13,
    35: 19,
    37: 26
}

# 押下カウント記録用
press_counts = {pin: 0 for pin in pin_map}

# GPIO初期化
GPIO.setmode(GPIO.BCM)
for gpio in pin_map.values():
    GPIO.setup(gpio, GPIO.IN, pull_up_down=GPIO.PUD_UP)

# コールバック関数
def button_callback(channel):
    for pin, gpio in pin_map.items():
        if channel == gpio:
            press_counts[pin] += 1
            clear_output(wait=True)
            output_lines = [f"### 🔘 ボタン押下情報"]
            for p in sorted(press_counts.keys()):
                output_lines.append(f"- 物理ピン {p}（GPIO {pin_map[p]}）: **{press_counts[p]} 回**")
            display(Markdown('\n'.join(output_lines)))
            break

# 割り込みの設定
for gpio in pin_map.values():
    GPIO.add_event_detect(gpio, GPIO.FALLING, callback=button_callback, bouncetime=300)

print("ボタン押下をカウント中...（Ctrl+C または GPIO.cleanup() で終了）")
